In [1]:
import os
import sys

import torch
import torch.nn as nn
import numpy as np

import onnx
from onnxsim import simplify
import onnxruntime

sys.path.append('../')

from siamfcpp.config.config_head import cfg as root_cfg
from siamfcpp.config.config_head import specify_task 
from siamfcpp.engine.builder import build as tester_builder
from siamfcpp.model import builder_head as model_builder
from siamfcpp.pipeline import builder as pipeline_builder
from siamfcpp.utils import complete_path_wt_root_in_cfg

from siamfcpp.model.task_head_new.taskhead_impl.track_head import get_box, get_xy_ctr

from platforms.utils.onnx_utils import load_onnx, run_onnx

In [2]:
config = '../models/siamfcpp/test/vot/siamfcpp_alexnet.yaml'
model_path = '../models/snapshots/siamfcpp_alexnet-got/epoch-17.pkl'

In [3]:
exp_cfg_path = os.path.realpath(config)
root_cfg.merge_from_file(exp_cfg_path)
root_cfg.test.track.model.task_model.SiamTrack.pretrain_model_path = model_path

# ROOT_PATH = os.getcwd() 
# root_cfg = complete_path_wt_root_in_cfg(root_cfg, ROOT_PATH)
root_cfg = root_cfg.test
task, task_cfg = specify_task(root_cfg)
task_cfg.freeze()

torch.multiprocessing.set_start_method('spawn', force=True)

In [4]:
model = model_builder.build("track", task_cfg.model)

2023-03-14 09:57:30.738 | INFO     | siamfcpp.model.module_base:update_params:60 - Load pretrained SiamTrack parameters from: ../models/snapshots/siamfcpp_alexnet-got/epoch-17.pkl whose md5sum is 2a050730626f1b083baed91f9a5c4a52


#### head

In [5]:
inp1 = torch.Tensor(np.random.uniform(size=(1,256,23,23)).astype(np.float32))
inp2 = torch.Tensor(np.random.uniform(size=(1,256,23,23)).astype(np.float32))

torch_model = model.head
torch_model.eval()

out = torch_model(inp1, inp2)

small_model_path = '../models/onnx/head_5.onnx'

torch.onnx.export(torch_model, 
                  (inp1,inp2), 
                  small_model_path, 
                  input_names=['input1', 'input2'], output_names=['csl_score', 'ctr_score', 'offsets', 'fea'],
                  verbose=True, opset_version=11) 

simplified_model, check = simplify(small_model_path, skip_fuse_bn=False)
onnx.save_model(simplified_model, small_model_path)

graph(%input1 : Float(1, 256, 23, 23, strides=[135424, 529, 23, 1], requires_grad=0, device=cpu),
      %input2 : Float(1, 256, 23, 23, strides=[135424, 529, 23, 1], requires_grad=0, device=cpu),
      %cls_p5_conv1.conv.weight : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %cls_p5_conv1.conv.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %bbox_p5_conv1.conv.weight : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %bbox_p5_conv1.conv.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %cls_p5_conv2.conv.weight : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %cls_p5_conv2.conv.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %bbox_p5_conv2.conv.weight : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %bbox_p5_conv2.conv.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %86 : 

In [6]:
np.random.seed(42)

inp1 = torch.Tensor(np.random.uniform(size=(1,256,23,23)).astype(np.float32))
inp2 = torch.Tensor(np.random.uniform(size=(1,256,23,23)).astype(np.float32))

out = torch_model(inp1, inp2)

onnx_model = load_onnx(small_model_path, 'CUDAExecutionProvider')
out_onnx = run_onnx(onnx_model, {'input1': inp1.numpy().copy(), 'input2': inp2.numpy().copy()})

onnx_default = load_onnx('../models/onnx/head.onnx', 'CUDAExecutionProvider')
out_default = run_onnx(onnx_default, {'input1': inp1.numpy().copy(), 'input2': inp2.numpy().copy()})

idx = 0

print(torch.sum(out[idx]-torch.Tensor(out_onnx[idx])))

tensor(-4.88758087e-05, grad_fn=<SumBackward0>)


In [7]:
offsets = out_onnx[2].copy()
offsets = np.exp(offsets*0.6790788769721985 + 1.6184018850326538) * 8
ctr = get_xy_ctr(17, 87, 8)
offset = get_box(ctr, torch.Tensor(offsets)).numpy()
out_onnx[2] = offset

In [13]:
idx = 2

print(np.sum(out_default[idx]-out_onnx[idx]))

0.00015258789
